In [89]:
import numpy as np
import os
import json
import sys
import matplotlib.pyplot as plt
import skimage

sys.path.append("../")
import src.plot as myplot
import seaborn as sns

import sklearn

In [90]:
import importlib
import src.plot as myplot
importlib.reload(myplot)

<module 'src.plot' from '/Users/paulhosek/PycharmProjects/mcm/MCM_classifier/Classifier_1/0_classifier/paper_plots/../src/plot.py'>

In [91]:

import importlib
import src.paper_utils as utils
importlib.reload(utils)

<module 'src.paper_utils' from '/Users/paulhosek/PycharmProjects/mcm/MCM_classifier/Classifier_1/0_classifier/paper_plots/../src/paper_utils.py'>

In [92]:
sample_sizes = [10, 100, 500,1000, 2000, 3000,4000,5000]
sample_sizes = [500]
counts_samplesizes, mcms_samplesizes = utils.load_counts_mcm(sample_sizes, letter="", path_format="../Output/sample_sizes")


In [93]:
n_runs = 2
mcm_digit = 0

nr_digits = 10
maxnr_icc = 121

test_probs = np.full((nr_digits, len(sample_sizes), n_runs,nr_digits,maxnr_icc, len(utils.load_test_data(digit=0))), fill_value=-1.0)
for mcm_digit in range(nr_digits):

    for test_digit in range(nr_digits):
        test_data = utils.load_test_data(digit=test_digit)

        for sample_size_idx, sample_size in enumerate(sample_sizes):

            mcms = mcms_samplesizes[sample_size_idx][:n_runs]
            counts_gstar = counts_samplesizes[sample_size_idx][:n_runs]
                
            for run_idx, mcm in enumerate(mcms):
                test_probs[mcm_digit][sample_size_idx][run_idx][test_digit] = utils.probabilities_gstar(mcm[mcm_digit], counts_gstar[run_idx][mcm_digit], test_data, sample_size,smooth=True,return_distr_icc=True)
                

[-1.00000000e+00  1.55938124e-05  3.11876248e-05  6.23752495e-05
  1.24750499e-04  2.49500998e-04  4.99001996e-04  2.01160180e-03
  2.02719561e-03  2.05838323e-03  2.12075848e-03  2.24550898e-03
  2.49500998e-03  4.00760978e-03  4.02320359e-03  4.05439122e-03
  4.11676647e-03  4.24151697e-03  6.00361776e-03  6.01921158e-03
  6.05039920e-03  6.11277445e-03  6.23752495e-03  6.98602794e-03
  7.99962575e-03  8.01521956e-03  8.04640719e-03  8.23353293e-03
  8.48303393e-03  9.99563373e-03  1.00112275e-02  1.00424152e-02
  1.01047904e-02  1.02295409e-02  1.04790419e-02  1.19916417e-02
  1.20072355e-02  1.20384232e-02  1.21007984e-02  1.24750499e-02
  1.39876497e-02  1.40032435e-02  1.40344311e-02  1.40968064e-02
  1.44710579e-02  1.59836577e-02  1.59992515e-02  1.60304391e-02
  1.79796657e-02  1.79952595e-02  1.80264471e-02  1.82135729e-02
  1.99756737e-02  1.99912675e-02  2.00224551e-02  2.00848303e-02
  2.19872754e-02  2.20184631e-02  2.24550898e-02  2.39676896e-02
  2.39832834e-02  2.40144

KeyboardInterrupt: 

In [ ]:
distr = utils.probabilities_gstar(mcm[mcm_digit], counts_gstar[run_idx][mcm_digit], test_data, sample_size,smooth=True, return_distr_icc=True)

np.unique(distr)

array([-1.00000000e+00,  1.55938124e-05,  3.11876248e-05,  6.23752495e-05,
        2.49500998e-04,  4.99001996e-04,  2.01160180e-03,  2.02719561e-03,
        2.05838323e-03,  2.49500998e-03,  4.00760978e-03,  4.02320359e-03,
        4.05439122e-03,  4.49101796e-03,  6.00361776e-03,  6.01921158e-03,
        6.05039920e-03,  6.23752495e-03,  7.99962575e-03,  8.01521956e-03,
        8.04640719e-03,  8.23353293e-03,  8.48303393e-03,  9.99563373e-03,
        1.00112275e-02,  1.00424152e-02,  1.19916417e-02,  1.20072355e-02,
        1.20384232e-02,  1.39876497e-02,  1.40032435e-02,  1.40344311e-02,
        1.42215569e-02,  1.44710579e-02,  1.59836577e-02,  1.59992515e-02,
        1.60304391e-02,  1.62175649e-02,  1.69660679e-02,  1.79796657e-02,
        1.79952595e-02,  1.80264471e-02,  1.99756737e-02,  1.99912675e-02,
        2.00224551e-02,  2.00848303e-02,  2.19872754e-02,  2.20184631e-02,
        2.24550898e-02,  2.39676896e-02,  2.40144711e-02,  2.59636976e-02,
        2.59792914e-02,  

In [ ]:
np.unique(res)

array([-1,  0])

# Find indicative ICC

In [ ]:
samples_path = "../OUTPUT/sample_sizes"
mcms_fname = "MCMs_0.json"
counts_fname = "Counts_0.json"
n_samples = 5423

with open(os.path.join(samples_path,str(n_samples),"MCMs", mcms_fname), "r") as f:
        all_MCM = [np.array(i) for i in json.load(f)] 


with open(os.path.join(samples_path,str(n_samples),"Counts", counts_fname), "r") as f:
        C = json.load(f)
mcm_index = 3

In [ ]:
# # n_variables = 121 #TODO This cell is not trash.
# # data_path = "../INPUT_all/data/"
# # data_filename_format = "train-images-unlabeled-{}.dat"

# # # # for all images in all categories, get the PDF of that all ICC of one MCM fitted to one image
# all_C_icc_3 = myplot.calculate_P_icc(C, all_MCM,3,n_variables, data_path,data_filename_format)
# np.save("../OUTPUT/sample_sizes/5423/all_C_icc_3.npy", all_C_icc_3)



In [ ]:
all_C_icc_3 = np.load("../OUTPUT/sample_sizes/5423/all_C_icc_3.npy")

mcm3_comms_map = myplot.generate_icc_comms_map(all_MCM[3])
mcm5_comms_map = myplot.generate_icc_comms_map(all_MCM[5])

In [ ]:
n_variables = 121
test_data_path = "../INPUT_all/data/testdata_separated"
test_data_filename_format = "test-images-unlabeled-{}.dat"
test_all_C_icc_3 = myplot.calculate_P_icc(C, all_MCM,3,n_variables, test_data_path,test_data_filename_format)


In [ ]:
cat_a = 3
cat_b = 5
mcm_idx = 3

all_prob = test_all_C_icc_3/all_C_icc_3.shape[1] # for MCM 0
avg_prob = all_prob.mean(axis=1)

comms3 = myplot.generate_icc_comms_map(all_MCM[3])
distances = utils.normalised_signed_distance_decisionbound(avg_prob, cat_a, cat_b)
dist_map = distances[comms3] # plt.scatter(avg_prob[:,cat_a], avg_prob[:,cat_b])

In [ ]:
avg_prob[0,:]

array([0.0429883 , 0.01272028, 0.00506872, 0.04457658, 0.01235437,
       0.03651529, 0.03008652, 0.0188309 , 0.02524582, 0.01544617])

In [ ]:

seedigit = 5

colors = myplot.get_list_fromcmap("tab10", N=10)
fig, axs = plt.subplots(2,len(sample_sizes)//2, figsize=(15,5),sharex=True, sharey=True)
axs = np.ravel(axs)
for ss_idx, sample_size in enumerate(sample_sizes):
    data = test_probs[:,ss_idx,0].T
    ax = axs[ss_idx]



    logbins = np.geomspace(data.min(),data.max(), 40)

    for mcm_idx in [3,5]:
        data = test_probs[mcm_idx,ss_idx,0, seedigit].T
        ax.hist(data,bins=logbins, facecolor="None",alpha=.2,edgecolor="black",histtype= "stepfilled")
        ax.hist(data,bins=logbins, label=f'{mcm_idx}',alpha=.5,histtype= "stepfilled",facecolor=colors[mcm_idx])
    ax.text(.1,.8,f"Our distance {utils.norm_distribution_distance(np.log(test_probs[0,ss_idx,0, seedigit]),np.log(test_probs[1,ss_idx,0, seedigit])).round(2)}", transform=ax.transAxes)
    ax.text(.1,.7,f"TVD {utils.total_variation_distance(np.log(test_probs[0,ss_idx,0, seedigit]),np.log(test_probs[1,ss_idx,0, seedigit])).round(2)}", transform=ax.transAxes)

    # if ss_idx == len(sample_sizes)-3:
    #     handles, labels = ax.get_legend_handles_labels()
    #     ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=5)

    ax.set_title(sample_size)
    ax.set_xscale("log")

handles, labels = ax.get_legend_handles_labels()
leg = fig.legend(handles, labels, loc='center right', ncol=1,bbox_to_anchor=(.95,0.5)) #bbox_to_anchor=(.95,0.5)
leg.set_title("MCM",)

fig.supxlabel(f"Probability for digit {seedigit} at g* on testset (logscale)")

supylabel = fig.supylabel('Frequency')
supylabel.set_x(supylabel.get_position()[0] + 0.05)



plt.show()


